In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [1]:
import psycopg2
from dotenv import load_dotenv
import os
import pandas as pd

In [2]:
# Ładujemy zmienne środowiskowe z pliku .env
load_dotenv()

# Pobieranie ścieżki do pliku SQL z pliku .env
sql_path = os.getenv('SQL_PATH')

# Sprawdź, czy zmienna została poprawnie wczytana
print(f"Ścieżka do pliku SQL: {sql_path}")

# Pobieranie danych z .env
DB_HOST = os.getenv('DB_HOST')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PORT = os.getenv('DB_PORT')
DB_USERNAME = os.getenv('DB_USERNAME')
DB_PASSWORD = os.getenv('DB_PASSWORD')

# Funkcja do nawiązania połączenia z bazą danych
def connect_to_db():
    try:
        # Nawiązywanie połączenia z bazą danych
        connection = psycopg2.connect(
            host=DB_HOST,
            database=DB_DATABASE,
            user=DB_USERNAME,
            password=DB_PASSWORD,
            port=DB_PORT
        )
        print("Połączono z bazą danych!")
        return connection
    except Exception as e:
        print(f"Błąd podczas połączenia z bazą danych: {e}")
        return None

# Połączenie z bazą danych
connection = connect_to_db()

# Jeśli połączenie się udało, możemy wykonać zapytania SQL
if connection:
    cursor = connection.cursor()
    
    # Przykład zapytania - wyświetlanie tabel w bazie danych
    cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public';")
    tables = cursor.fetchall()
    print("Tabele w bazie danych:", tables)
    
    cursor.close()
    connection.close()

Ścieżka do pliku SQL: C:\Users\joann\Desktop\Dajana_ds\repozytoria\SQL\nba.sql
Połączono z bazą danych!
Tabele w bazie danych: [('studenci',), ('nba_sql',), ('titanic_dataset_csv',)]


In [3]:
def execute_sql_file(filepath):
    try:
        with open(filepath, 'r') as file:
            sql_commands = file.read()

        # Podziel zawartość pliku na poszczególne zapytania SQL
        sql_commands_list = sql_commands.split(';')

        # Połączenie z bazą danych
        connection = connect_to_db()
        cursor = connection.cursor()

        # Wykonanie każdego zapytania SQL osobno
        for command in sql_commands_list:
            if command.strip():  # Sprawdź, czy zapytanie nie jest puste
                cursor.execute(command)
        
        # Zatwierdź zmiany
        connection.commit()
        print("Zapytania z pliku SQL zostały wykonane!")

        cursor.close()
        connection.close()
    except Exception as e:
        print(f"Błąd podczas wykonywania zapytań z pliku SQL: {e}")


In [4]:
def fetch_data_to_dataframe(query):
    try:
        # Połączenie z bazą danych
        connection = connect_to_db()
        
        # Wykonanie zapytania i zapisanie wyników do DataFrame
        df = pd.read_sql(query, connection)
        
        # Zamknięcie połączenia
        connection.close()
        
        return df
    except Exception as e:
        print(f"Błąd podczas pobierania danych: {e}")
        return None

# Zapytanie SQL
query = "SELECT * FROM nba_sql;"  # Zakładam, że masz tabelę nba_sql w swojej bazie danych

# Wywołanie funkcji i załadowanie danych do DataFrame
df_nba = fetch_data_to_dataframe(query)

# Wyświetlenie danych
print(df_nba)


Połączono z bazą danych!


C:\Users\joann\AppData\Local\Temp\ipykernel_20924\1483118248.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


       column1       player_name team_abbreviation   age  player_height  \
0            0  Randy Livingston               HOU  22.0         193.04   
1            1  Gaylon Nickerson               WAS  28.0         190.50   
2            2      George Lynch               VAN  26.0         203.20   
3            3    George McCloud               LAL  30.0         203.20   
4            4      George Zidek               DEN  23.0         213.36   
...        ...               ...               ...   ...            ...   
12839    12839       Joel Embiid               PHI  29.0         213.36   
12840    12840   John Butler Jr.               POR  20.0         213.36   
12841    12841      John Collins               ATL  25.0         205.74   
12842    12842      Jericho Sims               NYK  24.0         208.28   
12843    12843    JaMychal Green               GSW  33.0         205.74   

       player_weight                college   country draft_year draft_round  \
0          94.80072

In [5]:
df_nba.head()

,column1,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,0,Randy Livingston,HOU,22.0,193.04,94.800728,Louisiana State,USA,1996,2,...,3.9,1.5,2.4,0.3,0.042,0.071,0.169,0.487,0.248,1996-97
1,1,Gaylon Nickerson,WAS,28.0,190.50,86.182480,Northwestern Oklahoma,USA,1994,2,...,3.8,1.3,0.3,8.9,0.030,0.111,0.174,0.497,0.043,1996-97
2,2,George Lynch,VAN,26.0,203.20,103.418976,North Carolina,USA,1993,1,...,8.3,6.4,1.9,-8.2,0.106,0.185,0.175,0.512,0.125,1996-97
3,3,George McCloud,LAL,30.0,203.20,102.058200,Florida State,USA,1989,1,...,10.2,2.8,1.7,-2.7,0.027,0.111,0.206,0.527,0.125,1996-97
4,4,George Zidek,DEN,23.0,213.36,119.748288,UCLA,USA,1995,1,...,2.8,1.7,0.3,-14.1,0.102,0.169,0.195,0.500,0.064,1996-97
